# CLIP 基础实践

CLIP (Contrastive Language-Image Pre-training) 是 OpenAI 提出的多模态模型。

## 核心思想

- **Image Encoder**: 将图像编码成向量
- **Text Encoder**: 将文本编码成向量
- **训练目标**: 让匹配的图像-文本对在向量空间中更接近

## 主要用途

1. **零样本图像分类** - 不训练直接分类
2. **图像检索** - 根据文本找图像
3. **文本检索** - 根据图像找文本

## 1. 安装和导入

In [ ]:
# 安装依赖 (如果还没安装)
# !pip install torch torchvision transformers pillow matplotlib

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available: {torch.cuda.is_available()}")

## 2. 加载预训练模型

我们使用 `clip-vit-base-patch32`，这是 CLIP 的基础版本

In [ ]:
# 加载模型和处理器
# 这会下载约 400MB 的模型文件
print("正在加载模型...")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("模型加载完成！")

## 3. 准备示例图像

我们下载几张图片来测试

In [ ]:
# 下载示例图像
url1 = "https://images.pexels.com/photos/1108099/pexels-photo-1108099.jpeg"
url2 = "https://images.pexels.com/photos/116975/pexels-photo-116975.jpeg"

image1 = Image.open(requests.get(url1, stream=True).raw)
image2 = Image.open(requests.get(url2, stream=True).raw)

# 显示图像
print("图片1: 狗狗")
display(image1)
print("\n图片2: 猫咪")
display(image2)

## 4. 图像编码

把图像变成向量

In [ ]:
# 预处理图像
inputs = processor(images=[image1, image2], return_tensors="pt")

# 获取图像特征
image_features = model.get_image_features(**inputs)

print(f"图像特征形状: {image_features.shape}")
print("(2张图片，每张512维向量)")

## 5. 文本编码

把文本变成向量

In [ ]:
# 定义文本描述
texts = ["a photo of a dog", "a photo of a cat", "a photo of a car"]

# 预处理文本
text_inputs = processor(text=texts, return_tensors="pt")

# 获取文本特征
text_features = model.get_text_features(**text_inputs)

print(f"文本特征形状: {text_features.shape}")
print("(3个句子，每句512维向量)")

## 6. 计算相似度

通过向量相似度判断图像和文本的匹配程度

In [ ]:
# 归一化特征
image_features_normalized = image_features / image_features.norm(dim=-1, keepdim=True)
text_features_normalized = text_features / text_features.norm(dim=-1, keepdim=True)

# 计算相似度 (点积)
logits_per_image = image_features_normalized @ text_features_normalized.T
logits_per_text = logits_per_image.T

# 转换成概率
probs = logits_per_image.softmax(dim=-1)

print("图像-文本相似度概率:")
print(f"图片1 (狗狗): {probs[0].detach().numpy()}")
print(f"图片2 (猫咪): {probs[1].detach().numpy()}")

## 7. 零样本分类

让 CLIP 自己判断图片是什么

In [ ]:
# 定义类别标签
classes = ["dog", "cat", "car", "bird"]

# 构建文本描述
texts = [f"a photo of a {c}" for c in classes]

# 处理图像和文本
image_inputs = processor(images=[image1, image2], return_tensors="pt")
text_inputs = processor(text=texts, return_tensors="pt")

# 计算相似度
image_features = model.get_image_features(**image_inputs)
text_features = model.get_text_features(**text_inputs)

# 归一化并计算相似度
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = image_features @ text_features.T

# 找到最可能的类别
for i, img in enumerate([image1, image2]):
    top_probs, top_indices = similarity[i].softmax(dim=0).topk(2)
    print(f"\n图片 {i+1} 的预测:")
    for prob, idx in zip(top_probs, top_indices):
        print(f"  {classes[idx]}: {prob.item():.2%}")

## 8. 自己试试！

试着修改上面的代码：

1. 换不同的图片
2. 换不同的文本描述
3. 添加更多类别

In [ ]:
# 你的练习代码在这里
# 试试看能否正确识别...